In [1]:
import yaml

ModuleNotFoundError: No module named 'yaml'

In [ ]:
# Read configuration
with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

config


{'config_dummy': {'a': 2, 'b': 10, 'c': 256}}

In [ ]:
# Read master
with open("master.yaml", "r") as f:
    master = yaml.load(f, Loader=yaml.FullLoader)
    
master


{'gen1': {'script': ['do_operations', 'print_output']}}

In [ ]:
# Read recipes
with open("recipes.yaml", "r") as f:
    recipes = yaml.load(f, Loader=yaml.FullLoader)

recipes

{'do_operations': {'script': ['add', 'multiply', 'power', 'factorial']},
 'print_output': {'script': ['print_result']}}

In [ ]:
for gen in sorted(master.keys()):
    print("Now browsing generation: " + gen)
    for recipe in master[gen]['script']:
        print("Now browsing recipe: " + recipe)
        print(recipes[recipe]['script'])
    print()

Now browsing generation: gen1
Now browsing recipe: do_operations
['add', 'multiply', 'power', 'factorial']
Now browsing recipe: print_output
['print_result']



In [ ]:
# Try get function code
from src import power

print('*********')
print(power.get_str())
print('*********')
print(power.get_output_str())
print('*********')
print(power.get_output_type_hint_str())
print('*********')
print(power.get_call_str())
print('*********')
print(power.get_assignation_call_str())
print('*********')
print(power.get_signature())
print('*********')
print(power.parameters)
print('*********')
print(power.dict_imports)
print('*********')



*********
def power_function(b: int, c: int) -> int:
    # Returns a at the power of b
    return np.power(b, c)

*********
output_power
*********
int
*********
power_function(b, c)
*********
output_power = power_function(b, c)
*********
(b: int, c: int) -> int
*********
OrderedDict([('b', <class 'int'>), ('c', <class 'int'>)])
*********
{'numpy': 'np'}
*********


In [ ]:
# Generate all blocks in a file
import src.blocks as blocks
from jinja2 import Environment

# Generate all blocks
file_path = "output_jinja_test_gen1.py"
for gen in sorted(master.keys()):
    print("Now browsing generation: " + gen)
    print('\n')
    dic_blocks = {}
    for recipe in master[gen]['script']:
        print("Now browsing recipe: " + recipe)
        print('')
        for block in recipes[recipe]['script']:
            print("Now browsing block: " + block)
            dic_blocks[block] = getattr(blocks,block)
            
            
# Build template with all blocks separated by \n
dic_blocks_str = {k: v.get_str() for k,v in dic_blocks.items()}
str_jinja = "\n".join([f"{{{{{k}}}}}" for k in dic_blocks.keys()])
template = Environment().from_string(str_jinja)
with open(file_path, "w") as f:
    f.write(template.render(**dic_blocks_str))
                

Now browsing generation: gen1


Now browsing recipe: do_operations

Now browsing block: add
Now browsing block: multiply
Now browsing block: power
Now browsing block: factorial
Now browsing recipe: print_output

Now browsing block: print_result


In [ ]:
# Try merging blocks

# Add takes a and b and returns output_add
# Power takes b and c and returns output_power
# Let's add to output c instead of output add
from typing import OrderedDict

dic_blocks["add"].output = OrderedDict([("c", dic_blocks["add"].output["output_add"])])

add_power = dic_blocks["add"].merge_block(
    dic_blocks["power"],
    "add_power",
    docstring="Test with long docstring",
    output = dic_blocks["power"].output,
)

In [ ]:
print('*********')
print(add_power.get_str())
print('*********')
print(add_power.get_output_str())
print('*********')
print(add_power.get_output_type_hint_str())
print('*********')
print(add_power.get_call_str())
print('*********')
print(add_power.get_assignation_call_str())
print('*********')
print(add_power.get_signature())
print('*********')
print(add_power.parameters)
print('*********')
print(add_power.dict_imports)
print('*********')
print(add_power.set_deps)


*********
def add_power(a: int, b: int) -> int:
    """Test with long docstring
    """

    c = add_function(a, b)
    output_power = power_function(b, c)
    return output_power

*********
output_power
*********
int
*********
add_power(a, b)
*********
output_power = add_power(a, b)
*********
(a: int, b: int) -> int
*********
OrderedDict([('a', <class 'int'>), ('b', <class 'int'>)])
*********
OrderedDict([('numpy', 'np')])
*********
{'power_function', 'add_function'}


In [ ]:
print(dic_blocks["add"].output)

OrderedDict([('c', <class 'int'>)])


In [ ]:
# Now with several blocks
from src import Block

dic_blocks["add"].parameters = OrderedDict([("a", dic_blocks["add"].output["c"]), ("output_factorial", dic_blocks["factorial"].output["output_factorial"])])
dic_blocks["multiply"].output = OrderedDict([("c", dic_blocks["multiply"].output["output_multiply"])])
dic_blocks["power"].output = OrderedDict([("d", dic_blocks["power"].output["output_power"])])
dic_blocks["print_result"]

test_multiple = Block.merge_blocks(
    [dic_blocks["factorial"], dic_blocks["add"], dic_blocks["multiply"], dic_blocks["power"], dic_blocks["print_result"]],
    "test_multiple_blocks",
    docstring="This function contains many blocks",
    output = dic_blocks["power"].output,
)

In [ ]:
print('*********')
print(test_multiple.get_str())
print('*********')
print(dic_blocks["add"].get_str())
print('*********')
print(test_multiple.get_output_str())
print('*********')
print(test_multiple.get_output_type_hint_str())
print('*********')
print(test_multiple.get_call_str())
print('*********')
print(test_multiple.get_assignation_call_str())
print('*********')
print(test_multiple.get_signature())
print('*********')
print(test_multiple.parameters)
print('*********')
print(test_multiple.dict_imports)
print('*********')
print(test_multiple.set_deps)

*********
def test_multiple_blocks(a: int, b: int) -> int:
    """This function contains many blocks
    """

    output_factorial = gamma_function(a)
    c = add_function(a, output_factorial)
    c = multiply_function(a, b)
    d = power_function(b, c)
    print_result_function(d)
    return d

*********
def add_function(a: int, output_factorial: int) -> int:
    """Dummy docstring
    """
    b = output_factorial
    
    # Add a and b
    return a + b

*********
d
*********
int
*********
test_multiple_blocks(a, b)
*********
d = test_multiple_blocks(a, b)
*********
(a: int, b: int) -> int
*********
OrderedDict([('a', <class 'int'>), ('b', <class 'int'>)])
*********
OrderedDict([('math', 'math'), ('numpy', 'np')])
*********
{'add_function', 'multiply_function', 'gamma_function', 'power_function', 'print_result_function'}


In [ ]:
import blocks

: 

In [2]:
from blocks import dic_blocks
dic_blocks

{'save_npy': <study_gen.block.Block at 0x1096d3560>,
 'add': <study_gen.block.Block at 0x109720050>,
 'save_pkl': <study_gen.block.Block at 0x10ca87500>,
 'load_npy': <study_gen.block.Block at 0x10ce7f980>,
 'multiply': <study_gen.block.Block at 0x10ce7f5c0>,
 'print_result': <study_gen.block.Block at 0x10924af30>,
 'power': <study_gen.block.Block at 0x10cf09b50>,
 'gamma': <study_gen.block.Block at 0x10cf0b0e0>}

In [ ]:
blocks.gamma

<module 'blocks.gamma' from '/Users/macbe16879/Desktop/study-gen/prototyping/simple/blocks/gamma.py'>

: 